# Hyperparameter tuning

In de reeds besproken machine learning technieken hebben we reeds een aantal keer vermeld dat er hyperparameters (denk aan regularisatieparameters, manieren van regularisatie, kernel type, ...).

In het geval van lineaire regressie gaat het dan over:
* L1 of L2 norm
* Regularisatieparameter $\lambda$
* learning rate

In het geval van SVM over:
* Type kernel
* Regularisatieparameter C
* Regularisatieparameter $\gamma$

Tot nu bestond de zoektocht naar de optimale combinatie van deze parameters door het manueel uitproberen en evalueren van een reeks combinaties van parameters.
Deze methode is echter niet schaalbaar en kan geautomatiseerd worden.
Dit gebeurd door middel van een gridsearch.

## Gridsearch

Het gridsearch algoritme bestaat eruit om een lijst op te stellen voor elke parameter welke waarden moeten getest worden.
Voor elke mogelijke combinatie van parameters gaat er dan een model getrained en geevalueerd worden.
Een voorbeeld van hoe dit kan geautomatiseerd worden binnen sklearn kan [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) gevonden worden.

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
import numpy as np

iris = datasets.load_iris()

parameters = {'C': np.arange(1,10), 'kernel': ['linear', 'rbf'], 'gamma': np.arange(0.1,10,0.1)}

clf = GridSearchCV(svm.SVC(), parameters)

clf.fit(iris.data, iris.target) # hier niet gesplitst op train, test voor demo

GridSearchCV(estimator=SVC(),
             param_grid={'C': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'gamma': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9]),
                         'kernel': ['linear', 'rbf']})

In [ ]:
#clf.best_estimator_.predict(iris.data)
# analoog aan
clf.predict(iris.data) # standaard wordt het beste model genomen

clf.cv_results_ # bevat informatie over alle uitgevoerde runs en hun scores

clf.best_params_ # in mijn geval C=2, gamma=0.2 -> niet de kleinste waarden van de parameters die we overlopen hebben -> we moeten het dus niet opnieuw doen met een kleinere ondergrens

{'C': 2, 'gamma': 0.2, 'kernel': 'rbf'}

De standaard methode van hierboven gaat alle combinaties afgaan.
Andere methoden die sneller maar niet alle combinaties aftoetsen zijn
* [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)
* [HalvingGridSearchCv](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html#sklearn.model_selection.HalvingGridSearchCV)
* [HalvingRandomizedSearchCv](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html#sklearn.model_selection.HalvingRandomSearchCV)

Belangrijk om hierbij op te merken is dat het GridSearch algoritme enkel verschillende parameters van het model trained en dat er geen eigenschappen van de data kan veranderd worden.
Indien je ook een exhausieve search wilt doen van het aantal hogere orde features of de vorm van scaling die gebruikt wordt op input parameters. Moet je een eigen wrapper schrijven die nog deze zaken uittest en de performantie van de uiteindelijke modellen vergelijkt.

## Validatieset

Welke data kunnen we nu gebruiken om deze gridsearch te evalueren.
Zowel de testdata als de trainingsdata kan niet gebruikt worden omdat we niet kunnen evalueren op de data waarmee het model getrained is.
Om deze reden wordt de dataset typisch in drie opgedeeld, namelijk een training-, test- en validatieset.
De validatieset is de data die dan gebruikt kan worden voor hyperparameter tuning.
Typisch wordt de dataset dan in de volgende groottes opgedeeld:
* Testset: 15%
* Validatieset: 15% 
* Trainingsdata: 70%

Dit zijn echter geen vaste waarden en kunnen wat verschillen in de praktijk.
Hoe meer data je beschibaar is hoe groter het percentage trainingsdata kan zijn. 
In het geval van big-data applicaties kan dit oplopen tot 98%.

## K-fold cross validation

Bij het steeds gebruiken van dezelfde validatieset is het mogelijk dat er een unieke split is die leidt tot een onverwacht goed of slecht resultaat.
Om dit tegen te gaan kan er gebruik gemaakt worden van K-fold cross validation.
Daarbij berekenen we de verwachte error K keer, elke keer met een andere train en validatie set om zo de kans te verhogen dat het uiteindelijke model ook goed werkt op de testset met ongeziene data.
Standaard wordt er bij het gebruik van het gridsearch algoritme gebruik gemaakt van 5 folds voor het zoeken naar de beste hyperparameters.
Indien de standaard manier niet voldoet voor de gewenste toepassing kan je ook de split rechtstreeks uitvoeren.
Meer informatie over deze methode vind je [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

## Oefening

In [deze dataset](https://www.kaggle.com/mathchi/diabetes-data-set) is een hele reeks data beschikbaar over een aantal medische eigenschappen van personen en of deze personen diabetes hebben of niet.
Ga nu op zoek naar het beste model om te voorspellen of een persoon diabetes gaat hebben of niet.
Test hierbij zowel de logistische regressie en svm methoden en maak gebruik van gridsearch met 10-fold cross validation om de verschillende hyperparameters te testen. 

Wat is de hoogst behaalde accuraatheid en de benodigde hyperparameters?

Indien dit gelukt is, zoek ook het model dat de hoogste weighted f1-score behaald. 
Welke techniek gebruikte dit model en welke hyperparameters zijn er hiervoor gekozen?
Vergelijk beide modellen. Is er een significant verschil in de resulterende hyperparameters?
Is de behaalde accuraatheid sterk afwijkend?

In [9]:
import opendatasets as od
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [7]:
od.download("https://www.kaggle.com/mathchi/diabetes-data-set")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/mathchi/diabetes-data-set


100%|██████████| 8.91k/8.91k [00:00<00:00, 4.55MB/s]

In [13]:
df = pd.read_csv("./diabetes-data-set/diabetes.csv")
display(df.head())
y = df.Outcome
X = df.drop("Outcome", axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression())
])

params = {
    'clf__penalty': ['l1', 'l2', 'elasticnet'],
    'clf__C': np.arange(0.1,10,0.1),
    'clf__solver': ['saga']

}
gridsearch = GridSearchCV(pipeline, params, scoring='f1_weighted')
gridsearch.fit(X_train, y_train)
print(gridsearch.score(X_train, y_train), gridsearch.score(X_test, y_test))

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


0.7690691885623956 0.7574816487859967


c:\Users\jens.baetens3\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
495 fits failed out of a total of 1485.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
495 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\jens.baetens3\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\jens.baetens3\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

In [ ]:
params = {
    LogisticRegression(): {
        'clf__penalty': ['l1', 'l2', 'elasticnet'],
        'clf__C': np.arange(0.1,10,0.1),
        'clf__solver': ['saga']
    },
    SVC(): {
        'clf__C': np.arange(0.1,10,0.1)
    }
}

for ml in params:
    print(ml)
    print(params[ml])

    pipeline = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('clf', ml)
    ])

    gridsearch = GridSearchCV(pipeline, params[ml], scoring='f1_weighted')
    gridsearch.fit(X_train, y_train)


LogisticRegression()
{'clf__penalty': ['l1', 'l2', 'elasticnet'], 'clf__C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9]), 'clf__solver': ['saga']}
SVC()
{'clf__C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9,